# Games Data fetching from Sportradar and Supabase update

In [1]:
# Import necessary libraries
import os
import sys
from django.conf import settings
import pandas as pd
from datetime import datetime, date

In [2]:
def serialize_dates(obj):
    if isinstance(obj, date):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")

In [3]:
# Set up the Django environment
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "hooperstats.settings")
import django
django.setup()

In [4]:
# Import utility functions
from api.sportradar_utils import get_sportradar_data
from api.supabase_utils import get_supabase_client

In [5]:
# Get Supabase client
supabase = get_supabase_client()

In [6]:
# Fetch Phoenix Suns team ID from the teams table
suns_team = supabase.table('teams').select('*').eq('name', 'Phoenix Suns').execute()
suns_id = suns_team.data[0]['id']
suns_name = suns_team.data[0]['name']
print(f"Phoenix Suns ID: {suns_id}")
print(f"Phoenix Suns Name: {suns_name}")

Phoenix Suns ID: 583ecfa8-fb46-11e1-82cb-f4ce4684ea4c
Phoenix Suns Name: Phoenix Suns


In [7]:
# Fetch schedule data from Sportradar API
schedule_data = get_sportradar_data('games/2023/REG/schedule.json')

In [8]:
games_data = []
for game in schedule_data['games']:
    if game['home']['id'] == suns_id or game['away']['id'] == suns_id:
        games_data.append({
            'id': game['id'],
            'date': datetime.strptime(game['scheduled'][:10], '%Y-%m-%d').date(),
            'season': schedule_data['season']['year'],
            'home_team_id': game['home']['id'],
            'away_team_id': game['away']['id'],
            'home_team_name': game['home']['name'],
            'away_team_name': game['away']['name'],
            'home_points': game.get('home_points', 0),
            'away_points': game.get('away_points', 0),
        })

In [9]:
# Convert to DataFrame
df = pd.DataFrame(games_data)
df

,id,date,season,home_team_id,away_team_id,home_team_name,away_team_name,home_points,away_points
0,b06ae4da-247e-4696-973a-7f1cbd813fec,2023-10-25,2023,583ec825-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Golden State Warriors,Phoenix Suns,104,108
1,6b977a9a-8eec-44c3-85ca-ec8787d7dad0,2023-10-27,2023,583ecae2-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Los Angeles Lakers,Phoenix Suns,100,95
2,9393ca30-2192-4341-9770-20c95ff426b5,2023-10-29,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ece50-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,Utah Jazz,126,104
3,3d03bd2b-a31e-4b6d-8ffc-adae63a0e000,2023-11-01,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,San Antonio Spurs,114,115
4,f9f4b8f6-42fd-448a-a757-c3224c5201a1,2023-11-03,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,San Antonio Spurs,121,132
...,...,...,...,...,...,...,...,...,...
77,77f0a7d7-b408-4511-a07d-39834e592086,2024-04-07,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecc9a-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,New Orleans Pelicans,105,113
78,e0ceb8e3-13cd-41ce-8465-f53ce0c2ccff,2024-04-10,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecdfb-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,LA Clippers,92,105
79,f9998a42-9c4e-41b4-bd98-0c30012b331c,2024-04-11,2023,583ecdfb-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,LA Clippers,Phoenix Suns,108,124
80,3fa10377-ba83-4722-b5b8-0de05e23f861,2024-04-13,2023,583ed0ac-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Sacramento Kings,Phoenix Suns,107,108


In [10]:
# Sort by date and add game number
df = df.sort_values('date')
df['game_number'] = range(1, len(df) + 1)
df.head(4)

,id,date,season,home_team_id,away_team_id,home_team_name,away_team_name,home_points,away_points,game_number
0,b06ae4da-247e-4696-973a-7f1cbd813fec,2023-10-25,2023,583ec825-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Golden State Warriors,Phoenix Suns,104,108,1
1,6b977a9a-8eec-44c3-85ca-ec8787d7dad0,2023-10-27,2023,583ecae2-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Los Angeles Lakers,Phoenix Suns,100,95,2
2,9393ca30-2192-4341-9770-20c95ff426b5,2023-10-29,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ece50-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,Utah Jazz,126,104,3
3,3d03bd2b-a31e-4b6d-8ffc-adae63a0e000,2023-11-01,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,San Antonio Spurs,114,115,4


In [11]:
# Determine winner
df['winner_team_id'] = df.apply(lambda row: row['home_team_id'] if row['home_points'] > row['away_points'] else row['away_team_id'], axis=1)
df['winner_team_name'] = df.apply(lambda row: row['home_team_name'] if row['home_points'] > row['away_points'] else row['away_team_name'], axis=1)
df.head(4)

,id,date,season,home_team_id,away_team_id,home_team_name,away_team_name,home_points,away_points,game_number,winner_team_id,winner_team_name
0,b06ae4da-247e-4696-973a-7f1cbd813fec,2023-10-25,2023,583ec825-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Golden State Warriors,Phoenix Suns,104,108,1,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns
1,6b977a9a-8eec-44c3-85ca-ec8787d7dad0,2023-10-27,2023,583ecae2-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Los Angeles Lakers,Phoenix Suns,100,95,2,583ecae2-fb46-11e1-82cb-f4ce4684ea4c,Los Angeles Lakers
2,9393ca30-2192-4341-9770-20c95ff426b5,2023-10-29,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ece50-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,Utah Jazz,126,104,3,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns
3,3d03bd2b-a31e-4b6d-8ffc-adae63a0e000,2023-11-01,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,San Antonio Spurs,114,115,4,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,San Antonio Spurs


In [12]:
df['date'] = df['date'].astype(str)
df.head(4)

,id,date,season,home_team_id,away_team_id,home_team_name,away_team_name,home_points,away_points,game_number,winner_team_id,winner_team_name
0,b06ae4da-247e-4696-973a-7f1cbd813fec,2023-10-25,2023,583ec825-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Golden State Warriors,Phoenix Suns,104,108,1,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns
1,6b977a9a-8eec-44c3-85ca-ec8787d7dad0,2023-10-27,2023,583ecae2-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Los Angeles Lakers,Phoenix Suns,100,95,2,583ecae2-fb46-11e1-82cb-f4ce4684ea4c,Los Angeles Lakers
2,9393ca30-2192-4341-9770-20c95ff426b5,2023-10-29,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ece50-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,Utah Jazz,126,104,3,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns
3,3d03bd2b-a31e-4b6d-8ffc-adae63a0e000,2023-11-01,2023,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,San Antonio Spurs,114,115,4,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,San Antonio Spurs


In [13]:
import json

games_to_insert = json.loads(json.dumps(df.to_dict('records'), default=serialize_dates))

In [14]:
result = supabase.table('games').insert(games_to_insert).execute()

# Check the result
if result.data:
    print(f"Successfully inserted {len(result.data)} games.")
else:
    print("Failed to insert games.")
    print("Error:", result.error)

# Verify the insertion
final_games = supabase.table('games').select('*').execute()
print(f"\nFinal count of games in database: {len(final_games.data)}")

Successfully inserted 82 games.

Final count of games in database: 82
